In [1]:
!pip install tranformers
!pip install torch
!pip install sentencepiece

ERROR: Could not find a version that satisfies the requirement tranformers (from versions: none)
ERROR: No matching distribution found for tranformers
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 6.1 MB/s eta 0:00:00


In [2]:

from transformers import GPT2Tokenizer, GPT2LMHeadModel
from transformers import LlamaForCausalLM, LlamaTokenizer
from torch.nn.utils.rnn import pad_sequence
import torch
from torch.optim import AdamW
import pandas as pd
import numpy as np
import transformers
from transformers import LlamaForCausalLM, DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments
from transformers import pipeline
!pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 2.0 MB/s eta 0:00:00


In [3]:
#model_name = "gpt2"
#tokenizer = GPT2Tokenizer.from_pretrained(model_name)
#model = GPT2LMHeadModel.from_pretrained(model_name)

model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
tokenizer = transformers.AutoTokenizer.from_pretrained(model_name)
model = LlamaForCausalLM.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [4]:
df = pd.read_json("/content/syntheticdata.json")

inputs = []
for problems, solutions in zip(df.problem, df.solution):
  inputs.append("Problem: " + problems + " Solution: " + solutions)
targets = df.feedback

inputs, targets = np.array(inputs), np.array(targets)

In [5]:
df

,problem,solution,evaluation,feedback
0,The construction industry is a significant con...,Utilizing modular and reusable building materi...,"{'economic_stability': 8, 'technological_innov...",Great solution! Modular construction not only ...
1,The challenge of generating enough electrical ...,Investing in solar power farms and advancing e...,"{'economic_stability': 9, 'technological_innov...","Your focus on solar power is commendable, show..."
2,The increased carbon footprint from digital le...,Developing a platform for reusing electronic d...,"{'economic_stability': 7, 'technological_innov...","Your solution addresses the issue well, with g..."
3,The fashion industry is a top contributor to g...,Implementing sustainable and recyclable materi...,"{'economic_stability': 8, 'technological_innov...",Your emphasis on sustainable materials and cir...
4,Electronic devices are often replaced before t...,Designing electronic products with modular com...,"{'economic_stability': 9, 'technological_innov...",Your approach to modular components is excelle...
5,Businesses worldwide expend substantial resour...,Transitioning to digital receipts and promotin...,"{'economic_stability': 7, 'technological_innov...","Digital receipts are a good start, but conside..."
6,More than 130 billion plastic bottles are wast...,Implementing a comprehensive plastic recycling...,"{'economic_stability': 8, 'technological_innov...",Your focus on recycling and alternative packag...
7,"In congested cities like Berlin, finding parki...",Developing a smart parking system using IoT te...,"{'economic_stability': 9, 'technological_innov...","Your IoT-based solution is impressive, showing..."
8,The surplus of plastic waste is a major global...,Promoting the use of biodegradable plastics an...,"{'economic_stability': 8, 'technological_innov...",Your emphasis on biodegradable plastics and ad...
9,The usage of plastic bottles is a widespread e...,Encouraging the use of reusable water bottles ...,"{'economic_stability': 7, 'technological_innov...",Your focus on reusable bottles and bottle retu...


In [6]:
tokenizer.pad_token = tokenizer.eos_token
input_ids = tokenizer(inputs.tolist(), return_tensors="pt", padding = True, truncation=True).input_ids
target_ids = tokenizer(targets.tolist(), return_tensors="pt", padding = True, truncation=True).input_ids

In [7]:
max_length_input = max(len(tensor) for tensor in input_ids)
max_length_target = max(len(tensor) for tensor in target_ids)
max_length = max(max_length_input, max_length_target)

padded_input_ids = [torch.cat([tensor, torch.full((max_length - len(tensor),), tokenizer.pad_token_id)]) for tensor in input_ids]
padded_target_ids = [torch.cat([tensor, torch.full((max_length - len(tensor),), tokenizer.pad_token_id)]) for tensor in target_ids]

padded_input_ids = torch.stack(padded_input_ids)
padded_target_ids = torch.stack(padded_target_ids)

In [8]:
padded_target_ids.shape

torch.Size([31, 220])

In [9]:
print("Vocabulary size:", model.config.vocab_size)

Vocabulary size: 32000


In [10]:
print("BOS Token ID:", tokenizer.bos_token_id)
print("EOS Token ID:", tokenizer.eos_token_id)
print("PAD Token ID:", tokenizer.pad_token_id)

BOS Token ID: 1
EOS Token ID: 2
PAD Token ID: 2


In [ ]:
optimizer = AdamW(model.parameters(), lr=2e-5)
num_epochs = 4
for _ in range(num_epochs):
  for i in range(len(input_ids)):
      optimizer.zero_grad()

      input_batch = padded_input_ids[i].unsqueeze(0)  # Adds a batch dimension
      target_batch = padded_target_ids[i].unsqueeze(0)  # Adds a batch dimension

      outputs = model(input_ids=input_batch, labels=target_batch)
      loss = outputs.loss
      loss.backward()
      optimizer.step()

model.save_pretrained("fine_tuned_model")

In [ ]:
path = "/content/AI EarthHack Dataset.csv"
df = pd.read_csv(path, encoding = "ISO-8859-1")

index = 1
problem_example = "Evaluate this problem solution: Problem: " + df["problem"][index] + "Solution: "  + df["solution"][index]

In [ ]:
new_input_text = problem_example
new_input_ids = tokenizer.encode(new_input_text, return_tensors="pt")
new_output = model.generate(new_input_ids, pad_token_id=tokenizer.eos_token_id)
new_generated_text = tokenizer.decode(new_output[0], skip_special_tokens=True)
print(new_generated_text)

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1273: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1281: UserWarning: Input length of input_ids is 238, but `max_length` is set to 20. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.
  warnings.warn(


Evaluate this problem solution: Problem: I'm sure you, like me, are feeling the heat - literally! With World Health Organization declaring climate change as ""the greatest threat to global health in the 21st century"", we're in a race against time to move away from fossil fuels to more efficient, less polluting electrical power. But as we take bold leaps into a green future with electric cars and heating, we're confronted with a new puzzle - generating enough electrical power without using fossil fuels!  Solution: Imagine standing on a green hill, not a single towering, noisy windmill in sight, and yet, you're surrounded by wind power generation! Using existing, yet under-utilized technology, I propose a revolutionary approach to harness wind energy on a commercial scale, without those ""monstrously large and environmentally damaging windmills"". With my idea, we could start construction tomorrow and give our electrical grid the jolt it needs, creating a future where clean, quiet and e

In [ ]:
problem_example